In [6]:
# multithreading_tf_local.py
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor
import time
import psutil
import GPUtil
import os

# -----------------------------
# TensorFlow threading config
# -----------------------------
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)

# -----------------------------
# Local image path
# -----------------------------
IMAGE_PATH = "image.jpg"  # Make sure this exists in the current folder

# -----------------------------
# Function to load and process image
# -----------------------------
def load_and_process(_):
    img_bytes = tf.io.read_file(IMAGE_PATH)
    img = tf.io.decode_jpeg(img_bytes, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = tf.image.per_image_standardization(img)
    return img

# -----------------------------
# Function to check GPU usage
# -----------------------------
def get_gpu_usage():
    try:
        gpus = GPUtil.getGPUs()
        return [(gpu.id, gpu.load * 100, gpu.memoryUtil * 100) for gpu in gpus]
    except Exception:
        return []

# -----------------------------
# Measure start stats
# -----------------------------
cpu_start = psutil.cpu_percent(interval=None)
gpu_start = get_gpu_usage()
start_time = time.time()

# -----------------------------
# Run 15 threads in parallel
# -----------------------------
with ThreadPoolExecutor(max_workers=15) as executor:
    results = list(executor.map(load_and_process, range(15)))

# -----------------------------
# Measure end stats
# -----------------------------
end_time = time.time()
cpu_end = psutil.cpu_percent(interval=1)
gpu_end = get_gpu_usage()

# -----------------------------
# Print results
# -----------------------------
print("\n🧵 MULTITHREADING RESULTS (LOCAL IMAGE)")
print("======================================")
print(f"Time taken: {end_time - start_time:.3f} sec")
print(f"CPU usage start → end: {cpu_start:.1f}% → {cpu_end:.1f}%")
print("GPU usage (end):")
for gpu_id, load, mem in gpu_end:
    print(f"  GPU-{gpu_id}: {load:.1f}% load, {mem:.1f}% mem")
print(f"Processed {len(results)} images successfully.")



🧵 MULTITHREADING RESULTS (LOCAL IMAGE)
Time taken: 0.157 sec
CPU usage start → end: 11.0% → 8.0%
GPU usage (end):
  GPU-0: 0.0% load, 0.0% mem
Processed 15 images successfully.
